[참고 링크 : dscwinterstudy 님의 블로그](https://velog.io/@dscwinterstudy/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0-%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94-%EB%94%A5%EB%9F%AC%EB%8B%9D2-7%EC%9E%A5.-RNN%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%9C-%EB%AC%B8%EC%9E%A5-%EC%83%9D%EC%84%B1-5rk67hoiuv)
# Chapter07 : RNN을 사용한 문장 생성
__본 장에서는 앞선 5장과 6장에서 다룬 RNN과 LSTM을 통해 문장생성 수행__ <br>

구체적으로는 우선 말뭉치를 사용해 학습한 언어 모델을 이용하여 새로운 문장을 만들어 낸다. <br>
그런 다음 개선된 언어 모델을 이용하여 더 자연스러운 문장을 생성해보겠습니다. <br>
여기까지 해보면 AI로 글을 쓰게 한다는 개념을 실감할 수 있을 것이다. <br>

이 뿐 아니라 __seq2seq__ 라는 새로운 구조의 신경망도 다룰 예정이다. <br>
__seq2seq__란, from seq to seq를 뜻하는 말로, 한 시계열 데이터를 다른 시계열 데이터로 변환하는 걸 말한다. <br>

※ 이번 장에서는 RNN 두 개를 연결하는 아주 간단한 방법으로 seq2seq를 구현해 볼 것이다. <br>

이 seq2seq는 기계번역, 챗봇, 메일의 자동 답신 등 다양하게 응용될 수 있더. <br>
이번 절에서는 언어 모델로 문장을 생성해보겠다.

##### 7.1.1 RNN을 사용한 문장 생성의 순서

앞 장에서의 LSTM 계층을 이용하여 언어 모델을 구현했는데, <br>
그 모델의 신경망 구성은 다음 그림처럼 생겼다. <br>
그리고 시계열 데이터를 $T$개분 만큼 모아 처리하는 Time LSTM 과 Time Affine 계층 등을 만들었다.

![](img/fig-7-1.png)

이제 언어 모델에게 문장을 생성시키는 순서를 설명하겠다. <br>
이번에도 친숙한 "you say goodbye and I say hello"라는 말뭉치로 학습한 언어 모델을 예로 생각해보겠다. <br>

이 학습된 언어 모델로 "I"라는 단어를 입력으로 주면 어떻게 될꺼? <br>
그러면 이 언어 모델은 다음 그림과 같은 확률 분포를 출력한다고 한다.

![](img/fig-7-2.png)

언어 모델은 지금까지 주어진 단어들에서 다음에 출현하는 단어의 확률 분포를 출력한다 . <br>
이 결과를 기초로 다음 단어를 새로 생성하려면 어떻게 해야 할까? <br>

첫 번째로, 확률이 가장 높은 단어를 선택하는 방법을 떠올릴 수 있다. <br>
확률이 가장 높은 단어를 선택할 뿐 이므로, 결과가 일정하게 정해지는 결정적인 방법이다. <br>
또한, 확률적으로 선택하는 방법도 생각할 수 있다. <br>
각 후보 단어의 확률에 맞게 선택하는 것으로 확률이 높은 단어는 선택되기 쉽고, 확률이 낮은 단어는 선택되기 어려워진다. <br>
이 방식에서는 선택되는 단어(샘플링 단어)가 매번 다를 수 있다. <br>

우리는 매번 다른 문장을 생성하도록 하겠다. <br>
그 편이 생성되는 문장이 다양해져서 재미있을 것이다. 

> ![](img/fig-7-3.png) 
> <br>
> 그림은 확률분포로부터 샘플링을 수행한 결과로 say 가 선택된 경우를 보여준다 . <br>
> 다만 필연적이지는 않고(결정적이 아니고) 확률적으로 결정된다는 점에 주의한다. <br>
> 다른 단어들도 해당 단어의 출현 확률애 따라 정해진 비율만큼 샘플링될 가능성이 있다는 뜻이다. 


> 결정적이란 알고리즘의 결과가 하나로 정해지는 것, 결과가 예측 가능한 것을 말한다. <br>
> 예컨대 앞의 예에서 확률이 가장 높은 단어를 선택하도록 하면, 그 것은 결정적인 알고리즘이다. <br>
> 한편, 확률적인 알고리즘에서는 결과가 확률에 따라 정해진다. <br>
> 따라서 선택되는 단어는 실행헐 때마다 달라진다. 

그러면 계속해서 두 번째 단어를 샘플링해보자. <br>
즉, 방금 생성한 단어인 say를 언어 모델에 입력하여 다음 단어의 확률 분포를 얻는다. <br>
그런 다음 그 확률분포를 기초로 다음에 출현할 단어를 샘플링한다. 

![](img/fig-7-4.png)

다음은 이 작업을 원하는 만큼 반복한다.  <br>
그러면 새로운 문장을 생성할 수 있다. <br>

여기에서 주목할 것은 __이렇게 생성한 문장은 훈련 데이터에는 존재하지 않는, 말 그대로 새로운 문장이라는 것이다.__ <br>
왜냐하면 언어 모델은 훈련 데이터를 암기한 것이 아니라, 훈련 데이터에서 사용된 단어의 정렬 패턴을 학습한 것아가 때문이다.  <br>

만약 언어 모델이 말뭉치로부터 단어의 출현 패턴을 올바르게 학습할 수 있다면, 그 모델이 새로 생성하는 문장은 우리 인간에게도 자연스럽고 의미가 통하는 문장일 것으로 기대할 수 있다.

##### 7.1.2 문장 생성 구현
앞 장에서 구현한 RnnlmGen 클래스에 문장 생성 메서드를 추가합니다. 

In [1]:
# RnnlmGen 클래스의 구현

import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids = None, sample_size = 100):
        word_ids = [start_id]
        
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())
            
            sampled = np.random.choice(len(p), size = 1, p = p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
                
        return word_ids

##### 문장 생성 메소드

In [9]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('Rnnlm.pkl')

#start 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]


#문장 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt)

you used yesterday frustration the introduction of entertainment which would mafia seek the make a include industry director and spot welcomed cents the trademark of the situation who wants that they would be the columns to an promise.
 at the over-the-counter the administration corp. is like another investments in surgery pictures.
 there is going to be buck they do n't allowed to be weyerhaeuser tainted to vans.
 they both tramp a new executive said demand a debt of the creation.
 mr. common executive multinational 're little trinity and you benefit the successor kenneth.
 the investment


##### 7.1.3 더 좋은 문장으로

좋은 언어 모델이 있으면 좋은 문장을 기대할 수 있다. <br>
앞 장에서 더 좋은 언어 모델을 BetterRnnlm 라는 클래스로 구현했다. <br>
여기에 문장 생성 기능을 추가하겠다. <br>

이 모델을 한 단계 개선하고 한층 더 큰 말뭉치를 사용하면 더 자연스러운 문장을 생성해줄 것이다.

## 7.2 seq2seq
시계열 데이터는 많다. <br>
언어 데이터, 음성 데이터, 동영성 데이터는 모두 시계열 데이터이다. <br>
그리고 이러한 시계열 데이터를 또 다른 시계열 데이터로 변환하는 문제도 숱하게 생각할 수 있다. <br>
예컨대 기계 번역이나 음성 인식을 들 수 있다. <br>
그 외에도 챗봇처럼 대화하는 애플리케이션이나 컴파일러처럼 소스 코드를 기계어로 변환하는 작업도 생각해볼 수 있다. 

이처럼 입력과 출력이 시계열 데이터인 문제는 아주 많다. 지금부터 우리는 시계열 데이터를 다른 시계열 데이터로 변환하는 모델을 생각해볼 것이다. <br>

이를 위한 기법으로 여기에서는 2개의 RNN을 이용하는 seq2seq라는 방법을 살펴보겠다. 

##### 7.2.1  seq2seq의 원리
seq2seq를 ___Encoder Decoder 모델___이라고도 한다. <br>
여기에는 2개의 모듈, Encoder와 Decoder 가 등장한다. <br>
문자 그대로 Encoder는 입력데이터를 인코딩(부호화)하고 Decoder는 인코딩된 디코딩(복호화)한다. <br>

그럼 seq2seq의 구조를 구체적인 예를 들어 설명하겠다. <br>
우리말을 영어로 번역하는 예를 살펴보자. <br>
"나는 고양이로소이다" 문장을 "I am a cat"으로 번역해보자.

![](img/fig-7-5.png)

Encoder 가 "나는 고양이로소이다" 라는 출발어 문장을 인코딩한다. <br>
이어서 그 인코딩한 정보를 Decoder 에 전달하고, Decoder 가 도착어 문장을 생성한다. <br>
이 때 Encoder가 인코딩한 정보에는 번역에 필요한 정보가 조밀하게 응축되어 있다. <br>
Decoder는 조밀하게 응축된 정보를 바탕으로 도착어 문장을 생성하는 것이다. <br>

이것이 seq2seq의 전체 그림이다 . <br>
Encoder와 Decoder가 협력하여 시계열 데이터를 다른 시계열 데이터로 변환하는 것이다 . <br>
그리고 Encoder와 Decoder로 RNN을 사용할 수 있다. <br>

이제 전체 과정을 자세히 살펴보자. <br>
우선 Encoder의 처리에 집중해보자. 

> __※ Encoder의 계층은 다음과 같이 구성된다.__
> ![](img/fig-7-6.png)
> <br>
> Encoder는 RNN을 이용해 시계열 데이터 $h$라는 은닉 상태 벡터로 변환한다, <br>
> 지금 예에서는 RNN으로써 LSTM을 이용했지만, 단순히 RNN이나 GRU 등도 이용할 수 있다. <br>
> 그리고 여기에서는 우리말 문장을 단어 단위로 쪼개 입력한다고 가정한다. 

그런데 그림에서 Encoder 가 출력하는 벡터 $h$는 LSTM 계층의 마지막 은닉 상태이다. <br>
이 마지막 은닉 상태 $h$에 입력 문장(출력어)를 번역하는 데 필요한 정보가 인코딩된다. <br>
여기서 중요한 점은 LSTM의 은닉 상태 $h$는 고정 길이 벡터라는 사실이다. <br>
그래서 __인코딩 한다는 것은 결국 임의 길이의 문장을 고정 길이 벡터로 변환하는 작업이 된다.__

![](img/fig-7-9.png)

그렇다면 __Decoder__는 이 인코딩 된 벡터를 어떻게 요리하여 도착어 문장을 생성하는 것일까 ? <br>
앞 절에서 다룬 문장 생성 모델을 그대로 이용할 수 있다.


> ![](img/fig-7-8.png)
> <br>
> 그림과 같이 __Decoder__는 앞 절의 신경망과 완전히 같은 구성이다. <br>
> ___※ 단 한가지, LSTM 계층이 벡터 $h$를 입력받는 다는 점이 다르다.___ <br>
> 참고로, 앞 절의 언어 모델에서는 LSTM 계층이 아무것도 받지 않는다. (굳이 따지자면 은닉상태로 영벡터를 받았다고 할 수 있다.)  <br>
> 이처럼 단 하나의 사소한 차이가 평범한 언어 모델을 번역도 해낼 수 있는 Decoder로 탈바꿈시킨다. 

##### 다음은 Decoder와 Encoder를 연결한 계층 구성이다.
> ![](img/fig-7-9.png)
> <br>
> __※ seq2seq 는 LSTM 두 개로 구성된다.__ <br>
> &nbsp;&nbsp;&nbsp;(Encoder의 LSTM, Decoder의 LSTM) <br>
> 이때 LSTM 계층의 은닉 상태가 Encoder와 Decoder 를 이어주는 가교가 된다 . <br>
> 순전파 때는 Encoder 에서 인코딩된 정보가 LSTM 계층의 은닉 상태를 통해 Decoder에 전해진다. <br>
> 그리고 seq2seq 의 역전파때는 이 가교를 통해 기울기가 Decoder로부터 Encoder로 전해진다. 

##### 7.2.2 시계열 데이터 변환용 장난감 문제

지금부터 다룰 문제에 대해 설명하겠다. <br>
우리는 시계열 변환 문제의 예로 더하기를 다루었다. <br>
구체적으로는 57+5와 같은 문자열을 seq2seq 에 건내면 62라는 정답을 내놓도록 학습시킬 것이다. <br>
참고로 이와 같이 머신러닝을 평가하고자 만든 간단한 문제를 장난감 문제라고 한다.

![](img/fig-7-10.png)

덧셈은 우리 인간에게는 쉬운 문제이다. <br>
그러니 seq2seq는 덧셈에 대해(정확히는 덧셈의 논리에 대해)아무것도 모른다. <br>
seq2seq는 덧셈의 샘플로부터 거기서 사용되는 문자의 패턴을 학습한다. <br>
과연 이런 식으로 해서 덧셈의 규칙을 올바르게 학습할 수 있는걸까? <br>

- ___우리는 지금까지 word2vec이나 언어 모델 등에서 문자를 단어 단위로 분할해 왔다.___

하지만, 문장을 반드시 단어로 분할해야 하는 건 아니다. <br>
실제로 이번 문제에서는 단어가 아닌 문자 단위로 분할한다. <br>
문자 단위 분할이란, 예컨대 57 + 5가 입력되면, [5,7,+,5] 라는 리스트로 처리하는 걸 말한다. 

##### 7.2.3 가변 길이 시계열 데이터
우리는 덧셈을 문자 리스트로써 다루기로 했다. <br>
이때 주의할 점은 덧셈 문장(5, 7, +, 5)이나 그 대답의 문자 수(6,2)가 문제마다 다르다는 것이다. <br>

이처럼 이번 덧셈 문제에서는 샘플마다 데이터의 시간 방향 크기가 다르다. <br>
가변 길이 시계열 데이터를 다룬다는 뜻이다. <br>
__따라서 신경망 학습 시 미니배치 처리를 하려면 무언가 추가 노력이 필요하다.__
> 미니배치로 학습할 때는 다수의 샘플을 한꺼번에 처리한다. <br>
> 이때 한 미니배치에 속한 샘플들의 데이터 형상이 모두 똑같아야 한다.

가변 길이 시계열 데이터를 미니배치로 학습하기 위한 가장 단순한 방법은 패딩을 사용하는 것이다. <br>
패딩이란, 원래의 데이터에 의미없는 데이터를 채워 모단 데이터의 길이를 균일하게 맞추는 기법이다.



> 다음 그림은 패딩을 이번 덧셈 문제에 적용해본 모습이다. <br>
> 모든 입력 데이터의 길이를 통일하고, 남는 공간에는 의미없는 데이터(여기서는 공백)을 채웠다. 
> <br>
> ![](img/fig-7-11.png)

이번 문제에서는 0~999 사이의 숫자 2개만 더하기로 하겠다. <br>
따라서 + 까지 포함하면 입력의 최대 문자의 수는 7 <br>
자연스럽게 덧셈 결과는 최대 4문자이다.  <br>

다불어 정답 데이터에도 패딩을 수행해 모든 샘플 데이터의 길이를 통일한다. <br>
그리고 질문과 정답을 구분하기 위해 출력 앞에 구분자로 _를 붙이기도 한다.<br>
그 결과 출력 데이터는 총 5문자로 통일한다. <br>
※ 참고로, 이 구분자는 Decoder 에 문자열을 생성하라라고 알리는 신호로 사용된다.  <br>

이처럼 패딩을 적용해 데이터 크기를 통일시키면 가변 길이 시계열 데이터도 처리할 수 있다. <br>
그러나 원리는 존재하지 않던 패딩용 문자까지  seq2seq 가 처리하게 된다. <br>
따라서, 패딩을 적용해야 하지만 정확성이 중요하다면 seq2seq에 패딩 전용 처리를 추가해야 한다. <br>

예컨대 Decoder 에 입력된 데이터가 패딩이라면 손실의 결과에 반영하지 않도록 해야 한다. <br>
Softmax with Loss 계층에 마스크 기능을 추가해 해결할 수 있다. <br>

한편 Encoder에 입력된 데이터가 패딩이라면 LSTM 계층이 이전 시각의 입력을 그대로 출력하게 한다. <br>
즉, LSTM 계층은 마치 처음부터 패딩이 존재하지 않았던 것 처럼 인코딩 할 수 있다. <br>

이번 장에서는 이해 난이도를 낮추기 위해 패딩용 문자(공백 문자)도 특별히 구분하지 않고 일반 데이터처럼 다루겠더. 

##### 7.2.4 덧셈 데이터셋

https://velog.io/@dscwinterstudy/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0-%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94-%EB%94%A5%EB%9F%AC%EB%8B%9D2-7%EC%9E%A5.-RNN%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%9C-%EB%AC%B8%EC%9E%A5-%EC%83%9D%EC%84%B1-5rk67hoiuv